In [1]:
#Imports
import pandas as pd
from pandas.core import datetools
import numpy as np
from patsy import dmatrices

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [2]:
# Loading dataset and view a few records.
df = pd.read_csv('E:\\MYLEARN\\2-ANALYTICS-DataScience\\datasets\\loan.csv')

In [3]:
df.head()

,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,addr_state,dti,delinq_2yrs,revol_util,total_acc,bad_loan,longest_credit_length,verification_status
0,5000,36 months,10.65,10.0,RENT,24000.0,credit_card,AZ,27.65,0.0,83.7,9.0,0,26.0,verified
1,2500,60 months,15.27,0.0,RENT,30000.0,car,GA,1.00,0.0,9.4,4.0,1,12.0,verified
2,2400,36 months,15.96,10.0,RENT,12252.0,small_business,IL,8.72,0.0,98.5,10.0,0,10.0,not verified
3,10000,36 months,13.49,10.0,RENT,49200.0,other,CA,20.00,0.0,21.0,37.0,0,15.0,verified
4,5000,36 months,7.90,3.0,RENT,36000.0,wedding,AZ,11.20,0.0,28.3,12.0,0,7.0,verified


In [4]:
df.isnull().sum()

loan_amnt                   0
term                        0
int_rate                    0
emp_length               5804
home_ownership              0
annual_inc                  4
purpose                     0
addr_state                  0
dti                         0
delinq_2yrs                29
revol_util                193
total_acc                  29
bad_loan                    0
longest_credit_length      29
verification_status         0
dtype: int64

In [5]:
# drop cols where null values exists
df.dropna(inplace=True)


In [6]:
df.isnull().sum()

loan_amnt                0
term                     0
int_rate                 0
emp_length               0
home_ownership           0
annual_inc               0
purpose                  0
addr_state               0
dti                      0
delinq_2yrs              0
revol_util               0
total_acc                0
bad_loan                 0
longest_credit_length    0
verification_status      0
dtype: int64

In [7]:
df.dtypes

loan_amnt                  int64
term                      object
int_rate                 float64
emp_length               float64
home_ownership            object
annual_inc               float64
purpose                   object
addr_state                object
dti                      float64
delinq_2yrs              float64
revol_util               float64
total_acc                float64
bad_loan                   int64
longest_credit_length    float64
verification_status       object
dtype: object

In [8]:
# drop non-numeric cols
df = df._get_numeric_data() 

In [9]:
df.dtypes

loan_amnt                  int64
int_rate                 float64
emp_length               float64
annual_inc               float64
dti                      float64
delinq_2yrs              float64
revol_util               float64
total_acc                float64
bad_loan                   int64
longest_credit_length    float64
dtype: object

In [10]:
df.head()

,loan_amnt,int_rate,emp_length,annual_inc,dti,delinq_2yrs,revol_util,total_acc,bad_loan,longest_credit_length
0,5000,10.65,10.0,24000.0,27.65,0.0,83.7,9.0,0,26.0
1,2500,15.27,0.0,30000.0,1.00,0.0,9.4,4.0,1,12.0
2,2400,15.96,10.0,12252.0,8.72,0.0,98.5,10.0,0,10.0
3,10000,13.49,10.0,49200.0,20.00,0.0,21.0,37.0,0,15.0
4,5000,7.90,3.0,36000.0,11.20,0.0,28.3,12.0,0,7.0


In [11]:
# subset the dataframe
# df = df[['annual_inc','loan_amnt', 'dti']]

In [12]:
X_df = df.drop(['annual_inc'], axis=1)

In [13]:
X_df.columns

Index(['loan_amnt', 'int_rate', 'emp_length', 'dti', 'delinq_2yrs',
       'revol_util', 'total_acc', 'bad_loan', 'longest_credit_length'],
      dtype='object')

# Step 1: Run a multiple regression

In [14]:
%%capture

# gather features
features = "+".join(X_df.columns )

In [15]:
features

'loan_amnt+int_rate+emp_length+dti+delinq_2yrs+revol_util+total_acc+bad_loan+longest_credit_length'

In [16]:
# get y and X dataframes based on this regression:
y, X = dmatrices('annual_inc ~' + features, df, return_type='dataframe')

In [17]:
X.shape

(157996, 10)

# Step 2: Calculate VIF Factors

In [18]:
# For each X, calculate VIF and save in dataframe
vif = pd.DataFrame()

vif["VIF_Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

vif["feature"] = X.columns

# Step 3: Inspect VIF Factors

In [19]:
vif.sort_values(['VIF_Factor'], ascending=False).round(5)

,VIF_Factor,feature
0,20.84630,Intercept
2,1.32138,int_rate
7,1.30385,total_acc
6,1.23580,revol_util
9,1.23558,longest_credit_length
4,1.16700,dti
1,1.16602,loan_amnt
3,1.08746,emp_length
8,1.06598,bad_loan
5,1.04300,delinq_2yrs
